# 第8章：使用cuGraph处理图

<img src="images/chapter-08/cugraph_logo_2.png" style="width:600px;"/>

cuGRAPH是RAPIDS AI套件的一部分,提供了一组针对GPU性能优化的图分析算法。它支持各种图数据结构和算法,能够快速处理大规模图数据。

它允许在cuDF中的ETL任务和cuML中的机器学习任务之间无缝传递数据。


## 主要优势：
- 性能：加速您的图计算。
- 可扩展性：处理对于仅CPU解决方案来说不可行的大型数据集。
- 集成：轻松与其他RAPIDS库组合

## 使用NetworkX代码的cuGraph
cuGraph现在可以通过nx-cugraph作为NetworkX的后端使用。这种与NetworkX的集成工作为NetworkX用户提供了一个零代码更改选项,可以使用NVIDIA GPU和cuGraph加速他们现有的NetworkX代码。


## 有用的参考链接

cuGraph文档：https://docs.rapids.ai/api/cugraph/stable/ 

NetworkX引入使用NVIDIA cuGraph的零代码更改加速：https://developer.nvidia.com/blog/networkx-introduces-zero-code-change-acceleration-using-nvidia-cugraph/

NetworkX文档：https://networkx.org/ 

## 编码指南

### 前提条件
- CUDA兼容的GPU：确保您的系统有兼容的NVIDIA GPU。
- 软件：安装RAPIDS AI库,包括cuGRAPH。

### 通过Conda安装

``` 
conda create -n rapids-24.10 -c rapidsai -c conda-forge -c nvidia  \
    cudf=24.10 cugraph=24.10 python=3.12 'cuda-version>=12.0,<=12.5' 
```

### 通过Pip安装

```
pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.10.* cugraph-cu12==24.10.* 
```


### 通过Docker安装

``` 
docker run --gpus all --pull always --rm -it \
    --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
    nvcr.io/nvidia/rapidsai/base:24.10-cuda12.5-py3.12
```

In [ ]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.10.* cugraph-cu12==24.10.* 

### 验证安装

在Python中运行以下命令：

```
import cugraph
print(cugraph.__version__)
```

## 核心功能


1. 图创建
从各种格式创建图(边列表、邻接矩阵)。

2. 算法
主要算法包括：
    - PageRank
    - 连通分量
    - 最短路径
    - 社区检测

3. 可视化
与可视化库集成以进行图表示。

## 动手示例

### 示例1：创建图

从边列表创建一个简单的图。

边列表是表示图的一种简单方式。它由节点对组成,每对表示两个节点之间的连接(或边)。

在我们的示例中,我们将创建一个由以下边列表表示的小图：

节点0连接到节点1
节点0连接到节点2
节点1连接到节点2
节点2连接回节点0

In [ ]:
import cudf
import cugraph

# 创建示例边列表
edge_list = cudf.DataFrame({
    'src': [0, 0, 1, 2],
    'dst': [1, 2, 2, 0]
})

# 创建图
G = cugraph.Graph()
G.from_cudf_edgelist(edge_list, source='src', destination='dst')

如上所示,'src'中每个索引处的值对应于连接到'dst'中相同索引处值的源节点。

接下来,我们创建一个来自cuGRAPH的Graph类的实例。这个对象将保存我们的图结构。

要将边列表加载到我们的图对象中,我们使用from_cudf_edgelist方法。此方法需要指定DataFrame的哪些列表示源节点和目标节点。


### 可选：可视化图

- 安装所需库：NetworkX和Matplotlib
```
pip install networkx matplotlib
```

- 转换为NetworkX格式：

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

#创建空的NetworkX图

nx_graph = nx.DiGraph()


# 从cuGRAPH添加边到NetworkX
for u, v in zip(edge_list['src'].to_arrow().to_pylist(), edge_list['dst'].to_arrow().to_pylist()):
    nx_graph.add_edge(u, v)

# 可视化图

plt.figure(figsize=(8, 6))
pos = nx.spring_layout(nx_graph)  # 布局算法
nx.draw(nx_graph, pos, with_labels=True, node_color='lightblue', node_size=1000, font_size=15, font_weight='bold', arrows=True)
plt.title("使用NetworkX的图可视化")
plt.show()

### 💡 挑战：修改图
现在您可以可视化图了,试着修改边列表以创建一个更复杂的图并再次可视化它。不同的结构会如何改变布局？

现在我们已经配置了带有NetworkX的cuGraph设置,让我们开始使用真实世界的示例来试验它的功能！


## 🎬 使用cuGraph进行电影推荐

### 入门

在一个电影观众经常被选择困扰的饱和市场中,我们希望确保用户收到量身定制的建议,突出隐藏的宝藏并促进发现,最终提高观众满意度和参与度。cuGraph在电影推荐方面很有用,因为我们可以使用内置的推荐算法(如PageRank)根据用户过去的偏好和评分来推荐电影。

MovieLens数据集是一个丰富的电影评分和用户偏好集合,包含来自多样化用户群的数百万条评分,捕捉了个人如何与数千部电影互动的见解。该数据集不仅包括用户生成的评分,还包括电影的元数据,如类型、标题和发行年份,使其成为构建和测试推荐算法的综合资源。

让我们开始加载数据集！

#### 用户评分数据集

这个数据集累积了943个用户对1682部不同电影的100000个不同评分,每个用户至少评价了20部电影。

In [ ]:
import pandas as pd 

columns = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=columns)
df.head()

#### 电影信息数据集

电影数据集包含有关电影ID、标题、发行日期、类型等信息。我们特别感兴趣使用这个数据集在提出推荐后将电影ID匹配到标题。

In [ ]:
item_cols = ['movie_id','movie_title','release_date', 'video_release_date',
              'MDb_URL', 'unknown','Action','Adventure','Animation',
              'Childrens','Comedy','Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery','Romance','Sci-Fi',
              'Thriller','War', 'Western' ]

item_df = pd.read_csv('u.item', encoding= 'ISO-8859-1', sep = '|', names = item_cols)

item_df.head()

### 🕸️ 构建我们的图

让我们使用`nx.Graph()`构建一个二分图,其中一个分区中是个人用户节点,另一个分区中是所有唯一电影。然后我们添加从用户到他们评价的电影的边,连接实际评分。

In [ ]:

C = nx.Graph()
df['user_id'] = df['user_id'].apply(lambda x: str(x) + '_user')
df['item_id'] = df['item_id'].apply(lambda x: str(x) + '_item')
user_ids = df['user_id'].unique()
#user_ids = user_ids + "-user"  #转换为字符串以避免与user_ids重复

print(f"唯一用户数量：{len(user_ids)}")
item_ids =df['item_id'].unique()

print(f"唯一电影数量：{len(item_ids)}")


C.add_nodes_from(user_ids, bipartite = 0)
C.add_nodes_from(item_ids, bipartite = 1)

edges = [(row['user_id'], row['item_id'], {'rating': row['rating']}) for _, row in df.iterrows()]



C.add_edges_from(edges)


print(f"节点数量：{C.number_of_nodes()}")
print(f"边数量：{C.number_of_edges()}")


从上面的输出可以得出一些结论。

我们有：
- 943个唯一电影观众对1682部唯一电影进行评分
- 每个943个唯一观众在图的`bipartite = 0`中代表一个节点
- 每个1682部唯一电影在`bipartite = 1`中是一个节点
- 所有用户和电影之间总共有100000个评分,对应于边的数量
- 每个评分代表一条边。

### 🕸️ 运行PageRank算法

在电影推荐领域,利用PageRank等算法可以显著提升用户体验。PageRank最初是为网页排名开发的,它分析基于用户互动的电影之间的关系,创建偏好网络。通过优先考虑不仅受欢迎而且通过用户评分和观看习惯相互关联的电影,PageRank可以提供更细致和相关的建议。

哪些电影最受欢迎？

让我们使用`nx.pagerank()`计算PageRank分数。


In [ ]:
pagerank_scores = nx.pagerank(C)
pagerank_df = cudf.DataFrame({'node_id': pagerank_scores.keys(), 'score': pagerank_scores.values()})
pagerank_df.tail()

如您所见,每个项目都标记了它们的分数。让我们过滤出每个项目的分数并从高到低排序。这里显示了排名前10的电影：

In [ ]:
item_scores_df = pagerank_df[pagerank_df['node_id'].str.endswith('item')]
sorted_scores_df = item_scores_df.sort_values(by='score', ascending=False).head(10)
sorted_scores_df

现在让我们使用之前加载的item_df找出它们各自的电影标题。

In [ ]:
top_10_movies = []
for _, row in sorted_scores_df.to_pandas().iterrows():
    movie_id = int(row['node_id'].split('_')[0])
    movie_title = item_df[item_df['movie_id'] == movie_id].iloc[0]['movie_title']
    top_10_movies.append(movie_title)

top_10_movies

## 结论

在本教程中,我们探索了cuGraph的强大功能,这是一个为高效图分析设计的GPU加速库。我们从环境设置和必要库的导入开始,然后转向加载和构建各种数据格式的图结构。

我们深入研究了关键算法,如PageRank和社区检测,演示了如何将这些技术应用于真实世界的数据集。通过利用cuGraph处理大规模图的能力,我们展示了与传统基于CPU的方法相比使用GPU加速的显著性能优势。

当您继续探索cuGraph时,考虑探索其他算法和功能,以及将图分析集成到更大的数据处理管道中。潜在的应用范围很广,从社交网络分析到推荐系统等等。

我们希望本教程为您提供了有效使用cuGraph的基础知识和技能。祝您图分析愉快！